In [1]:
import json
import numpy as np
import pandas
#import os
#import io
from pprint import pprint


In [2]:
with open('events.json','r') as f:
    allgames = json.load(f)
print('ok')

ok


In [3]:
nEvents = 0
print('{0:,.0f} games processed'.format(len(allgames)))
for game in allgames:
    nEvents = nEvents + len(game['events'])
print('{0:,.0f} events found'.format(nEvents))

152,259 games processed
14,946,418 events found


In [5]:
weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

nGames = 0
for game in allgames:
    if (game['id'] in weird_game_ids):
        nGames = nGames + 1
        for j in range(0,len(game['events'])):
            for k in range(0,len(game['events'][j])):
                print(k, game['events'][j][k])
        print('-----------')
print(nGames)        

#events_array = np.empty((0,11))
#nGames = 0
#for game in allgames:
#    nGames = nGames + 1
#    if (np.mod(nGames,100) == 0):
#        print('Processing game {0:,.0f}...'.format(nGames))
#    for j in range(0,len(game['events'])):
#        events_array = np.vstack((weird_events_array,game['events'][j]))
#print('Loaded into array: {0:,.0f} events'.format(len(events_array)))

0 1
1 play
2 1
3 0
4 bishm102
5 ??
6 
7 31
8 
9 
10 
0 2
1 play
2 1
3 0
4 hales101
5 ??
6 
7 9
8 
9 
10 
0 3
1 play
2 1
3 0
4 frenw101
5 ??
6 
7 3/G
8 
9 
10 
0 4
1 play
2 1
3 1
4 jamic101
5 ??
6 
7 43
8 
9 
10 
0 5
1 play
2 1
3 1
4 fonsl101
5 ??
6 
7 K23
8 
9 
10 
0 6
1 play
2 1
3 1
4 summh101
5 ??
6 
7 7
8 
9 
10 
0 7
1 play
2 2
3 0
4 cobbt101
5 ??
6 
7 23/BG
8 
9 
10 
0 8
1 play
2 2
3 0
4 cochm101
5 ??
6 
7 S8
8 
9 
10 
0 9
1 play
2 2
3 0
4 dykej101
5 ??
6 
7 D7.1-3
8 
9 
10 
0 10
1 play
2 2
3 0
4 wheaz101
5 ??
6 
7 K
8 
9 
10 
0 11
1 play
2 2
3 0
4 gallc101
5 ??
6 
7 W
8 
9 
10 
0 12
1 play
2 2
3 0
4 walbr101
5 ??
6 
7 43
8 
9 
10 
0 13
1 play
2 2
3 1
4 burng102
5 ??
6 
7 3/L
8 
9 
10 
0 14
1 play
2 2
3 1
4 sewej101
5 ??
6 
7 W
8 
9 
10 
0 15
1 play
2 2
3 1
4 sewel101
5 ??
6 
7 S8/THH.1XH(863452E5)(NR)(UR);B-H(NR)(UR)#
8 
9 
10 
0 16
1 com
2 
3 
4 
5 
6 
7 
8 "$Luke Sewell rounded 1B too far and Joe Sewell tried to score"
9 
10 
0 17
1 com
2 
3 
4 
5 
6 
7 
8 "during the rundown"
9

In [ ]:
weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

events_array = np.empty((0,12))
nGames = 0
for game in allgames:
    if (game['id'] in weird_game_ids):
        nGames = nGames + 1
        print('Processing game {0:,.0f}...'.format(nGames))
        for j in range(0,len(game['events'])):
            this_event_array = np.empty((1,12))
            this_event_array[0] =  game['events'][j]
#            this_event_array = np.vstack((this_event_array,game['id']))
        print(this_event_array.shape)
        events_array = np.vstack((events_array,this_event_array))           
#            game['events'][j].append(game['id'])  # add game ID to each row of events
#            events_array = np.vstack((events_array,game['events'][j]))
print('Loaded into array: {0:,.0f} events!'.format(len(events_array)))

events_df = pandas.DataFrame(events_array)

events_df.index.name = 'event_id'
column_list = ['event_in_game','event_type','inning','visitor_or_home','batter_id','pitch_count','pitch_list']
column_list += ['event','comment','sub_batting_order','sub_position']
events_df.columns = column_list
events_df

In [ ]:
events_df[events_df['comment'] != '']